In [17]:
%load_ext autoreload
%autoreload 2
import torch
import pytorch_lightning as pl
import torchvision.models as models

from brainage.model.agemodel_2dslices import AgeModel2DSlices

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
ckpt = '/home/raheppt1/projects/brainage/brainage/model/outputs/2020-07-14/20-19-10/my-project/3k717qdf/checkpoints/epoch=58.ckpt'

In [19]:
model = AgeModel2DSlices.load_from_checkpoint(ckpt)

TypeError: __init__() missing 2 required positional arguments: 'train_ds' and 'val_ds'

In [10]:
list(ckpt)

['epoch',
 'global_step',
 'pytorch-lightning_version',
 'checkpoint_callback_best_model_score',
 'checkpoint_callback_best_model_path',
 'optimizer_states',
 'lr_schedulers',
 'state_dict',
 'hparams_name',
 'hyper_parameters']

In [12]:
ckpt['state_dict']

.1041,
                       0.0048, -0.0880, -0.0495, -0.0327, -0.0287, -0.0555, -0.0113, -0.0557,
                      -0.0141, -0.0388, -0.0341, -0.0350, -0.0347, -0.0528, -0.0506, -0.0026,
                      -0.0240, -0.0130, -0.0154, -0.0328, -0.0064, -0.0635, -0.0304, -0.0150,
                      -0.0040, -0.0191, -0.0822, -0.0013, -0.0166, -0.0175, -0.0155, -0.0074,
                      -0.0467, -0.0040, -0.0155, -0.0325, -0.0030, -0.0374, -0.0273, -0.0044,
                      -0.0150, -0.0188, -0.0119, -0.0628, -0.0093, -0.0576, -0.0455, -0.0091,
                      -0.0600, -0.0816, -0.0327, -0.0247, -0.0184, -0.0235, -0.0311, -0.0510,
                      -0.0246, -0.0038, -0.0101, -0.0598, -0.0408, -0.0283, -0.0284, -0.0605,
                      -0.0497, -0.0351, -0.0269, -0.0273, -0.0201, -0.0614, -0.0204, -0.0437,
                      -0.0360, -0.0087, -0.0535, -0.0548, -0.0842, -0.0169, -0.0109, -0.0392,
                      -0.0346, -0.1356, -0.0420, -0.0